In [4]:
# If you're using Google Colab and not running locally, run this cell.

## Install dependencies
!pip install wget
!apt-get install sox libsndfile1 ffmpeg
!pip install unidecode
!pip install pytorch_lightning
!pip install python-Levenshtein


## Install NeMo
!python -m pip install nemo_toolkit[all]

# !pip install torchtext==0.10.1
# !pip install matplotlib==3.1.3
"""
Remember to restart the runtime for the kernel to pick up any upgraded packages (e.g. matplotlib)!
Alternatively, you can uncomment the exit() below to crash and restart the kernel, in the case
that you want to use the "Run All Cells" (or similar) option.
"""
exit() 

  Created wheel for wget: filename=wget-3.2-py3-none-any.whl size=9672 sha256=be1e44fe57a7fee2a41c0e0e859573afd475152a01273e0a2687cc06131fbb16
  Stored in directory: /root/.cache/pip/wheels/a1/b6/7c/0e63e34eb06634181c63adacca38b79ff8f35c37e3c13e3c02
Successfully built wget
Reading package lists... Done
Building dependency tree       
Reading state information... Done
libsndfile1 is already the newest version (1.0.28-4ubuntu0.18.04.2).
ffmpeg is already the newest version (7:3.4.8-0ubuntu0.2).
The following additional packages will be installed:
  libmagic-mgc libmagic1 libopencore-amrnb0 libopencore-amrwb0 libsox-fmt-alsa
  libsox-fmt-base libsox3
Suggested packages:
  file libsox-fmt-all
The following NEW packages will be installed:
  libmagic-mgc libmagic1 libopencore-amrnb0 libopencore-amrwb0 libsox-fmt-alsa
  libsox-fmt-base libsox3 sox
0 upgraded, 8 newly installed, 0 to remove and 37 not upgraded.
Need to get 760 kB of archives.
After this operation, 6,717 kB of additional disk s

In [1]:
# все import'ы
# NeMo's "core" package
import nemo
# NeMo's ASR collection - this collections contains complete ASR models and
# building blocks (modules) for ASR
import nemo.collections.asr as nemo_asr
import glob
import os
import subprocess
import tarfile
import wget
import json
import pytorch_lightning as pl
from Levenshtein import distance
import zipfile
import pandas as pd


[NeMo W 2021-11-30 12:30:14 optimizers:50] Apex was not found. Using the lamb or fused_adam optimizer will error out.
################################################################################
### WARNING, path does not exist: KALDI_ROOT=/mnt/matylda5/iveselyk/Tools/kaldi-trunk
###          (please add 'export KALDI_ROOT=<your_path>' in your $HOME/.profile)
###          (or run as: KALDI_ROOT=<your_path> python <your_script>.py)
################################################################################



In [5]:
data_dir = '/content'

In [6]:
! ls /content

drive	  farfield_dropped.csv	test	  train_manifest.jsonl
farfield  sample_data		test.tar


In [4]:
# Загрузка датасета Golos
def load(golos_url, fname):
  if not os.path.exists(os.path.join(data_dir, fname)):
      golos_path = wget.download(golos_url, os.path.join(data_dir, fname))
      print(f"Dataset downloaded at: {golos_path}")
  else:
      print("Tarfile already exists.")
      golos_path = os.path.join(data_dir, fname)
  return golos_path

golos_path = load("https://sc.link/Kqr", "test.tar")

if not os.path.exists(os.path.join(data_dir, '/test/')):
    tar = tarfile.open(golos_path)
    tar.extractall(path=data_dir)

Dataset downloaded at: /content/test.tar


In [7]:
# --- Building Manifest Files --- #

# Function to build a manifest
def build_manifest(manifest_rel, manifest_abs):
    manifest_path = os.path.split(os.path.abspath(manifest_rel))[0]
    with open(manifest_rel, 'r') as fin:
        with open(manifest_abs, 'w') as fout:
            for line in fin:
                metadata = json.loads(line)
                metadata["audio_filepath"]=os.path.join(manifest_path, metadata["audio_filepath"])
                json.dump(metadata, fout)
                fout.write('\n')
                
# Building Manifests
print("******")
train_rel = os.path.join(data_dir, 'test/farfield/manifest.jsonl')
train_abs = os.path.join(data_dir, 'test/farfield/farfield.jsonl')
if not os.path.isfile(train_abs):
  build_manifest(train_rel, train_abs)
test_manifest = train_abs
print("test_manifest", test_manifest)

train_rel = os.path.join(data_dir, 'test/crowd/manifest.jsonl')
train_abs = os.path.join(data_dir, 'test/crowd/crowd.jsonl')
if not os.path.isfile(train_abs):
  build_manifest(train_rel, train_abs)
train_manifest = train_abs
print("train_manifest", train_manifest)

******
test_manifest /content/test/farfield/farfield.jsonl
train_manifest /content/test/crowd/crowd.jsonl


In [8]:
#загрузка модели
def load(golos_url, fname):
  if not os.path.exists(os.path.join(data_dir, fname)):
      golos_path = wget.download(golos_url, os.path.join(data_dir, fname))
      print(f"Dataset downloaded at: {golos_path}")
  else:
      print("Tarfile already exists.")
      golos_path = os.path.join(data_dir, fname)
  return golos_path
  
load("https://sc.link/ZMv", "QuartzNet15x5_golos.nemo")
asr_model = nemo_asr.models.EncDecCTCModel.restore_from(os.path.join(data_dir, "QuartzNet15x5_golos.nemo"))

Dataset downloaded at: /content/QuartzNet15x5_golos.nemo


[NeMo W 2021-11-30 12:33:29 modelPT:131] If you intend to do training or fine-tuning, please call the ModelPT.setup_training_data() method and provide a valid configuration file to setup the train data loader.
    Train config : 
    manifest_filepath: train/golos_and_mcv.jsonl
    sample_rate: 16000
    labels:
    - ' '
    - а
    - б
    - в
    - г
    - д
    - е
    - ж
    - з
    - и
    - й
    - к
    - л
    - м
    - н
    - о
    - п
    - р
    - с
    - т
    - у
    - ф
    - х
    - ц
    - ч
    - ш
    - щ
    - ъ
    - ы
    - ь
    - э
    - ю
    - я
    batch_size: 64
    trim_silence: false
    max_duration: 20.0
    min_duration: 0.1
    num_workers: 20
    shuffle: true
    is_tarred: false
    tarred_audio_filepaths: null
    tarred_shard_strategy: scatter
    parser: ru
    
[NeMo W 2021-11-30 12:33:29 modelPT:138] If you intend to do validation, please call the ModelPT.setup_validation_data() or ModelPT.setup_multiple_validation_data() method and provide a

[NeMo I 2021-11-30 12:33:29 features:265] PADDING: 16
[NeMo I 2021-11-30 12:33:29 features:282] STFT using torch
[NeMo I 2021-11-30 12:33:43 save_restore_connector:149] Model EncDecCTCModel was successfully restored from /content/QuartzNet15x5_golos.nemo.


In [9]:
#создание обучающего объекта
trainer = pl.Trainer(gpus=1, max_epochs=3, precision=16) 

Using 16bit native Automatic Mixed Precision (AMP)
GPU available: True, used: True
TPU available: False, using: 0 TPU cores
IPU available: False, using: 0 IPUs


In [10]:
#изменение параметров

params_model = asr_model._cfg

# --- Config Information ---#
# try:
#     from ruamel.yaml import YAML
# except ModuleNotFoundError:
#     from ruamel_yaml import YAML
# config_path = '/usr/local/lib/python3.7/dist-packages/nemo/collections/asr/models/configs/'

# yaml = YAML(typ='safe')
# with open(config_path) as f:
#     params = yaml.load(f)
# print(params)
params_model['train_ds']['parser']='base'
params_model['validation_ds']['parser']='base'
params_model['test_ds']['parser']='base'
params_model['train_ds']['manifest_filepath'] = 'train_manifest.jsonl'
params_model['train_ds']['batch_size']=16
params_model['validation_ds']['manifest_filepath'] = test_manifest
params_model['validation_ds']['batch_size']=16
params_model['test_ds']['manifest_filepath'] = test_manifest
params_model['test_ds']['batch_size']=8
params_model['optim']['lr'] = 0.0001

In [11]:
#создание экземпляра
from omegaconf import DictConfig


first_asr_model = nemo_asr.models.EncDecCTCModel(cfg=DictConfig(params_model), trainer=trainer)

[NeMo I 2021-11-30 12:37:16 collections:173] Dataset loaded with 601 files totalling 0.64 hours
[NeMo I 2021-11-30 12:37:16 collections:174] 0 files were filtered totalling 0.00 hours


[NeMo W 2021-11-30 12:37:16 nemo_logging:349] /usr/local/lib/python3.7/dist-packages/torch/utils/data/dataloader.py:481: UserWarning: This DataLoader will create 20 worker processes in total. Our suggested max number of worker in current system is 2, which is smaller than what this DataLoader is going to create. Please be aware that excessive worker creation might get DataLoader running slow or even freeze, lower the worker number to avoid potential slowness/freeze if necessary.
      cpuset_checked))
    


[NeMo I 2021-11-30 12:37:16 collections:173] Dataset loaded with 1916 files totalling 1.41 hours
[NeMo I 2021-11-30 12:37:16 collections:174] 0 files were filtered totalling 0.00 hours
[NeMo I 2021-11-30 12:37:16 collections:173] Dataset loaded with 1916 files totalling 1.41 hours
[NeMo I 2021-11-30 12:37:16 collections:174] 0 files were filtered totalling 0.00 hours
[NeMo I 2021-11-30 12:37:16 features:265] PADDING: 16
[NeMo I 2021-11-30 12:37:16 features:282] STFT using torch


In [12]:
# Start training!!!
trainer.fit(first_asr_model)

LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]
[NeMo W 2021-11-30 12:37:19 modelPT:471] The lightning trainer received accelerator: <pytorch_lightning.accelerators.gpu.GPUAccelerator object at 0x7f475ae35510>. We recommend to use 'ddp' instead.


[NeMo I 2021-11-30 12:37:19 modelPT:561] Optimizer config = Novograd (
    Parameter Group 0
        amsgrad: False
        betas: [0.9, 0.98]
        eps: 1e-08
        grad_averaging: False
        lr: 0.0001
        weight_decay: 0.001
    )
[NeMo I 2021-11-30 12:37:19 lr_scheduler:752] Scheduler "<nemo.core.optim.lr_scheduler.CosineAnnealing object at 0x7f4759c8a390>" 
    will be used during training (effective maximum steps = 114) - 
    Parameters : 
    (warmup_steps: 500
    warmup_ratio: null
    min_lr: 0.0
    last_epoch: -1
    max_steps: 114
    )



  | Name              | Type                              | Params
------------------------------------------------------------------------
0 | preprocessor      | AudioToMelSpectrogramPreprocessor | 0     
1 | encoder           | ConvASREncoder                    | 18.9 M
2 | decoder           | ConvASRDecoder                    | 34.9 K
3 | loss              | CTCLoss                           | 0     
4 | spec_augmentation | SpectrogramAugmentation           | 0     
5 | _wer              | WER                               | 0     
------------------------------------------------------------------------
18.9 M    Trainable params
0         Non-trainable params
18.9 M    Total params
37.859    Total estimated model params size (MB)


Validation sanity check: 0it [00:00, ?it/s]

[NeMo W 2021-11-30 12:37:20 nemo_logging:349] /usr/local/lib/python3.7/dist-packages/torch/utils/data/dataloader.py:481: UserWarning: This DataLoader will create 20 worker processes in total. Our suggested max number of worker in current system is 2, which is smaller than what this DataLoader is going to create. Please be aware that excessive worker creation might get DataLoader running slow or even freeze, lower the worker number to avoid potential slowness/freeze if necessary.
      cpuset_checked))
    
[NeMo W 2021-11-30 12:37:21 patch_utils:50] torch.stft() signature has been updated for PyTorch 1.7+
    Please update PyTorch to remain compatible with later versions of NeMo.
[NeMo W 2021-11-30 12:37:22 nemo_logging:349] /usr/local/lib/python3.7/dist-packages/nemo/collections/asr/parts/submodules/jasper.py:280: UserWarning: __floordiv__ is deprecated, and its behavior will change in a future version of pytorch. It currently rounds toward 0 (like the 'trunc' function NOT 'floor'). T

Training: 0it [00:00, ?it/s]

[NeMo W 2021-11-30 12:39:34 nemo_logging:349] /usr/local/lib/python3.7/dist-packages/torch/optim/lr_scheduler.py:134: UserWarning: Detected call of `lr_scheduler.step()` before `optimizer.step()`. In PyTorch 1.1.0 and later, you should call them in the opposite order: `optimizer.step()` before `lr_scheduler.step()`.  Failure to do this will result in PyTorch skipping the first value of the learning rate schedule. See more details at https://pytorch.org/docs/stable/optim.html#how-to-adjust-learning-rate
      "https://pytorch.org/docs/stable/optim.html#how-to-adjust-learning-rate", UserWarning)
    


Validating: 0it [00:00, ?it/s]

Validating: 0it [00:00, ?it/s]

Validating: 0it [00:00, ?it/s]

In [ ]:
df = pd.read_json('prob.jsonl', lines = True)
paths = list(df['audio_filepath'])
hypo = asr_model.transcribe(paths2audio_files=paths)
df['hypothesis'] = hypo

Transcribing:   0%|          | 0/2564 [00:00<?, ?it/s]

[NeMo W 2021-11-29 19:01:59 patch_utils:50] torch.stft() signature has been updated for PyTorch 1.7+
    Please update PyTorch to remain compatible with later versions of NeMo.
[NeMo W 2021-11-29 19:01:59 nemo_logging:349] /usr/local/lib/python3.7/dist-packages/nemo/collections/asr/parts/submodules/jasper.py:280: UserWarning: __floordiv__ is deprecated, and its behavior will change in a future version of pytorch. It currently rounds toward 0 (like the 'trunc' function NOT 'floor'). This results in incorrect rounding for negative values. To keep the current behavior, use torch.div(a, b, rounding_mode='trunc'), or for actual floor division, use torch.div(a, b, rounding_mode='floor').
      ) // self.conv.stride[0] + 1
    


In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
leve = []

for i in range(len(list(df['text']))):
  leve.append(distance( list(df['text'])[i], list(df['hypothesis'])[i] ))

df['leve'] = leve
df

,id,audio_filepath,text,duration,hypothesis,leve
0,b99cc98e6ebdd2975af0050682a948f4,farfield/b99cc98e6ebdd2975af0050682a948f4.opus,фильм бамблби смотреть онлайн в хорошем качестве,3.206875,фильм бамблби смотреть онлайн в хорошем качестве,0
1,ada90cee7a7c273bb6331dd3a3e972ea,farfield/ada90cee7a7c273bb6331dd3a3e972ea.opus,включи другую музыку,2.282625,включи другую музыку,0
2,96a3206b07a50ba90f85d11a3604b0c1,farfield/96a3206b07a50ba90f85d11a3604b0c1.opus,афина что такое инвестиционный пай,3.284000,афина что такое инвестиционный пай,0
3,9f22684c432c13fcf6fc96cf16bf6cf7,farfield/9f22684c432c13fcf6fc96cf16bf6cf7.opus,сбер какое число будет в следующую среду,2.852062,сбер какое число будет в следующую среду,0
4,70d05224987f66a014a4bb4fda0ace2e,farfield/70d05224987f66a014a4bb4fda0ace2e.opus,на каком основании вы считаете что вы обезьяна...,4.623375,на каком основании вы считаете что вы обезьяна...,0
...,...,...,...,...,...,...
10249,30cf1a197021534389ed971ce5164417,farfield/30cf1a197021534389ed971ce5164417.opus,афина когда сбербанк был основан,6.866812,афина когда сбербанк был основан,0
10250,83fd94cd83001e30c8e3f8c3ffdf7f09,farfield/83fd94cd83001e30c8e3f8c3ffdf7f09.opus,джой когда я смогу забрать новую карту которую...,3.622250,джой когда я смогу забрать новую карту которую...,0
10251,dc2bfb03b0a1902fc0421067a4408c78,farfield/dc2bfb03b0a1902fc0421067a4408c78.opus,сколько занимает бон,2.138813,сколько занимает бонн,1
10252,ba1cff3411a64594e37fdf8b00335543,farfield/ba1cff3411a64594e37fdf8b00335543.opus,джой в какой битве прославился британский флот,4.987875,джой в какой битве прославился британский флот,0


In [ ]:
df = df.sort_values(by='leve', ascending=False) 
#del df['Unnamed: 0'] <- возникало при чтении файла из сохраненного csv
del df['hypothesis']
del df['leve']
df.to_json('train_manifest.jsonl', orient='records', lines=True)
df

,id,audio_filepath,text,duration
5070,02431856f67b9c4b0d56d39947cf4ef3,farfield/02431856f67b9c4b0d56d39947cf4ef3.opus,салют хочу посмотреть интересную веселую смешн...,5.059500
4527,3e03ebf8c56841662d47a9c088f84c3d,farfield/3e03ebf8c56841662d47a9c088f84c3d.opus,салют для шестнадцати семнадцатилетних детей ж...,6.027188
8833,0c116246523fdd4a7823bcc0fa29a7e8,farfield/0c116246523fdd4a7823bcc0fa29a7e8.opus,сбер переключить на канал телевидения пятый канал,4.554875
2552,059020210f4e155a2501ccc38ab7578f,farfield/059020210f4e155a2501ccc38ab7578f.opus,сбер какие новгородская земля ты знаешь,4.223500
5553,14396ba930c88527a3302cad541a3c7b,farfield/14396ba930c88527a3302cad541a3c7b.opus,джой запиши меня на стрижку в ближайшей к рабо...,4.511500
...,...,...,...,...
7148,2b9f591da8d6f4c5a8087c4aafd60af2,farfield/2b9f591da8d6f4c5a8087c4aafd60af2.opus,сбер братан мне сказали что ты женщина это правда,4.373875
3445,f8c964f44adba934f245359f0d780e1f,farfield/f8c964f44adba934f245359f0d780e1f.opus,афина хочу записаться на маник,3.346687
9394,34bbe26093a949754581ae8f3fbf94f6,farfield/34bbe26093a949754581ae8f3fbf94f6.opus,сбер организации государственный секретарь сша,3.250437
916,a8fcbfc2000bee72bd81627e1c7ffadb,farfield/a8fcbfc2000bee72bd81627e1c7ffadb.opus,сбер что нужно сделать чтобы быстрее забеременеть,4.801063


In [ ]:
df = df.drop(df[df.leve == 0].index)
#удалить все те, которые без ошибок; сильных выбросов не было, поэтому верхнюю границу не ставила

In [ ]:
df.to_csv('drive/MyDrive/farfield_dropped.csv')

In [3]:
#уже загрузила с диска в /content
df = pd.read_csv('farfield_dropped.csv')
del df['Unnamed: 0']
del df['hypothesis']
del df['leve']
df.to_json('train_manifest.jsonl', orient='records', lines=True)
df

,id,audio_filepath,text,duration
0,02431856f67b9c4b0d56d39947cf4ef3,farfield/02431856f67b9c4b0d56d39947cf4ef3.opus,салют хочу посмотреть интересную веселую смешн...,5.059500
1,3e03ebf8c56841662d47a9c088f84c3d,farfield/3e03ebf8c56841662d47a9c088f84c3d.opus,салют для шестнадцати семнадцатилетних детей ж...,6.027188
2,0c116246523fdd4a7823bcc0fa29a7e8,farfield/0c116246523fdd4a7823bcc0fa29a7e8.opus,сбер переключить на канал телевидения пятый канал,4.554875
3,059020210f4e155a2501ccc38ab7578f,farfield/059020210f4e155a2501ccc38ab7578f.opus,сбер какие новгородская земля ты знаешь,4.223500
4,14396ba930c88527a3302cad541a3c7b,farfield/14396ba930c88527a3302cad541a3c7b.opus,джой запиши меня на стрижку в ближайшей к рабо...,4.511500
...,...,...,...,...
596,7f861804e11e4270032896e5931d71d2,farfield/7f861804e11e4270032896e5931d71d2.opus,салют по чем нынче валюта в анапе по курсу цб,4.807563
597,6e78786f511b2ee9a55636f801afdc0d,farfield/6e78786f511b2ee9a55636f801afdc0d.opus,найти банкомат в радиусе один километр,4.172313
598,346fc79aa3becd3d6da6a261b55b3dd5,farfield/346fc79aa3becd3d6da6a261b55b3dd5.opus,афина малефисенту смотрел в высоком качестве о...,4.210750
599,7ccd492d73298a3edf50bb8e669c4b9b,farfield/7ccd492d73298a3edf50bb8e669c4b9b.opus,джой найти фильм список шиндлера,4.233562


In [ ]:
#загрузила из golos_opus в /content
#заменила относительные пути на абсолютные с помощью notepad++
#оставляем только farfield
df = pd.read_json('100hours.jsonl', lines = True)
paths = df['audio_filepath']
for p in range(len(paths)):
  if paths[p].startswith('crowd'):
    paths[p] = 0
df = df.drop(df[df.audio_filepath == 0].index)
df.to_json('prob.jsonl', orient='records', lines=True)

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  import sys


In [1]:
#кусок архива golos_opus

with zipfile.ZipFile('/content/drive/MyDrive/farfield_opus/farfield.zip', 'r') as zip_ref:
    zip_ref.extractall('')

In [13]:
first_asr_model.cuda()

# We will be computing Word Error Rate (WER) metric between our hypothesis and predictions.
# WER is computed as numerator/denominator.
# We'll gather all the test batches' numerators and denominators.
wer_nums = []
wer_denoms = []

# Loop over all test batches.
# Iterating over the model's `test_dataloader` will give us:
# (audio_signal, audio_signal_length, transcript_tokens, transcript_length)
# See the AudioToCharDataset for more details.
for test_batch in first_asr_model.test_dataloader():
        test_batch = [x.cuda() for x in test_batch]
        targets = test_batch[2]
        targets_lengths = test_batch[3]        
        log_probs, encoded_len, greedy_predictions = first_asr_model(
            input_signal=test_batch[0], input_signal_length=test_batch[1]
        )
        # Notice the model has a helper object to compute WER
        first_asr_model._wer.update(greedy_predictions, targets, targets_lengths)
        _, wer_num, wer_denom = first_asr_model._wer.compute()
        first_asr_model._wer.reset()
        wer_nums.append(wer_num.detach().cpu().numpy())
        wer_denoms.append(wer_denom.detach().cpu().numpy())

        # Release tensors from GPU memory
        del test_batch, log_probs, targets, targets_lengths, encoded_len, greedy_predictions

# We need to sum all numerators and denominators first. Then divide.
print(f"WER = {sum(wer_nums)/sum(wer_denoms)}")

[NeMo W 2021-11-30 12:52:31 nemo_logging:349] /usr/local/lib/python3.7/dist-packages/nemo/collections/asr/parts/submodules/jasper.py:280: UserWarning: __floordiv__ is deprecated, and its behavior will change in a future version of pytorch. It currently rounds toward 0 (like the 'trunc' function NOT 'floor'). This results in incorrect rounding for negative values. To keep the current behavior, use torch.div(a, b, rounding_mode='trunc'), or for actual floor division, use torch.div(a, b, rounding_mode='floor').
      ) // self.conv.stride[0] + 1
    


WER = 1.0003579098067288
